In [8]:
import pandas as pd
import numpy as np
import requests, json
import sys
import os
import pandas as pd



In [2]:
client_id = os.environ['BNET_CLIENT_ID']
client_secret = os.environ['BNET_SECRET']

def create_access_token(client_id, client_secret, region = 'us'):
    data = { 'grant_type': 'client_credentials' }
    response = requests.post('https://%s.battle.net/oauth/token' % region, data=data, auth=(client_id, client_secret))
    return response.json()

token = create_access_token(client_id, client_secret)['access_token']

In [76]:
members = pd.read_csv('steamed_hams.csv')
members

,char_name,char_id,char_level,char_class,last_login,date_diff,active_30_days,active_60_days
0,beardzdk,203791547,70,6,2023-11-14 11:44:13,-2 days +12:58:53.481541,True,True
1,beardzz,203426723,70,2,2023-11-10 09:57:34,-6 days +11:12:14.481541,True,True
2,beheadme,202008718,70,1,2023-11-05 12:01:11,-11 days +13:15:51.481541,True,True
3,bellfulgur,133925404,70,7,2023-11-15 13:37:56,-1 days +14:52:36.481541,True,True
4,blessedthot,202005152,70,2,2023-11-14 10:48:03,-2 days +12:02:43.481541,True,True
...,...,...,...,...,...,...,...,...
84,yenoslas,184606459,70,7,2023-11-02 13:47:22,-14 days +15:02:02.481541,True,True
85,zymbodia,201607820,70,1,2023-10-19 09:25:39,-28 days +10:40:19.481541,True,True
86,zymss,198791240,70,5,2023-11-06 15:09:55,-10 days +16:24:35.481541,True,True
87,zymsz,202566996,70,13,2023-11-06 15:09:08,-10 days +16:23:48.481541,True,True


In [107]:
profs_dict = {}

def profs(char_list):
    for char in char_list:
        try:
            req = requests.get(f'https://us.api.blizzard.com/profile/wow/character/barthilas/{char}/achievements/statistics?namespace=profile-us&locale=en_US&access_token={token}').json()

            df = pd.json_normalize(req.get('categories'))
            df = df[df['name'] == 'Skills']['sub_categories'][4][1]

            df2 = pd.json_normalize(df['statistics'])
            df2 = df2[df2['name'].isin([
                'Total Crafting Orders Fulfilled',
                'Public Crafting Orders Fulfilled',
                'Guild Crafting Orders Fulfilled',
                'Personal Crafting Orders Fulfilled'
                ])][['name','quantity']]

            profs_dict[char] = {
                'Total':df2[df2['name'] == 'Total Crafting Orders Fulfilled']['quantity'].iloc[0],
                'Public':df2[df2['name'] == 'Public Crafting Orders Fulfilled']['quantity'].iloc[0],
                'Guild':df2[df2['name'] == 'Guild Crafting Orders Fulfilled']['quantity'].iloc[0],
                'Personal':df2[df2['name'] == 'Personal Crafting Orders Fulfilled']['quantity'].iloc[0]
            
            
                }
        except:
            pass

    return profs_dict

#df2
professions_result = profs(members['char_name'])


In [110]:
professions_result

{'brakyn': {'Total': 208.0, 'Public': 15.0, 'Guild': 17.0, 'Personal': 176.0},
 'darknthicc': {'Total': 169.0,
  'Public': 91.0,
  'Guild': 5.0,
  'Personal': 73.0},
 'dingboy': {'Total': 444.0,
  'Public': 67.0,
  'Guild': 116.0,
  'Personal': 261.0},
 'enosslas': {'Total': 134.0, 'Public': 7.0, 'Guild': 87.0, 'Personal': 40.0},
 'extraineous': {'Total': 33.0, 'Public': 1.0, 'Guild': 8.0, 'Personal': 24.0},
 'fcw': {'Total': 81.0, 'Public': 2.0, 'Guild': 9.0, 'Personal': 70.0},
 'titrainium': {'Total': 205.0,
  'Public': 84.0,
  'Guild': 21.0,
  'Personal': 100.0},
 'undeadned': {'Total': 173.0,
  'Public': 17.0,
  'Guild': 89.0,
  'Personal': 67.0}}

In [115]:
pd.DataFrame(professions_result).T.sort_values('Total', ascending=False).to_csv('Professions_Leaderboard.csv')